In [15]:
import requests
import pandas as pd
import os

# El mantra de las visualizaciones
from matplotlib import pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go

In [2]:
foods = ["beef","pigmeat","sheepAndGoat","rawMilk","dairy","fruitAndVegetable","cereal","rice","oilseeds","sugar","oliveOil","wine"]

In [3]:
foods = ["beef","pigmeat","sheepAndGoat","rawMilk","dairy","fruitAndVegetable","cereal","rice","oilseeds","sugar"]

In [4]:
data={}
date = []
price = []
prod = []
var = []
for food in foods:
    url = f"https://www.ec.europa.eu/agrifood/api/{food}/prices?memberStateCodes=ES&beginDate=01/01/2001"
    resp = requests.get(url).json()

    for i in resp:
        try:
            date.append(i['beginDate'])
        except: #Rice doesn't have beginDate but has ym which means year month
            date.append(i['ym'])
        price.append(i['price'][1:])
        try:
            var.append(i['productName'])
        except:
            var.append("unknown")
        prod.append(food)
    data['product'] = prod
    data['date'] = date
    data['price'] = price
    data['variety'] = var

In [5]:
data = pd.DataFrame(data)
data.date = pd.to_datetime(data.date)
data.index = data.date
data = data.sort_index()

In [18]:
if not os.path.exists("../mydata"):
    os.makedirs("../mydata")
data.to_csv("mydata/data.csv")

In [7]:
beef = {}
bdate = []
bprice = []
for i, row in data.iterrows():
    if row['product'] == 'beef':
        bdate.append(row['date'])
        bprice.append(row['price'])
    
    beef['product'] = 'Beef'
    beef['date'] = bdate
    beef['price'] = bprice
beef = pd.DataFrame(beef)

In [8]:
beef.index = pd.to_datetime(beef.date)
beef.drop('date',axis=1,inplace=True)

In [9]:
beef.price = [float(i.replace(",",".")) for i in beef.price]

In [11]:
gbeef = pd.DataFrame(beef.groupby(['date'])["price"].mean())

In [12]:
px.line(gbeef)